In [2]:
from vmad import autooperator
from vmad.lib import linalg
from vmad.contrib.chisquare import MPIChiSquareProblem, MPIVectorSpace
from abopt.abopt2 import Preconditioner
from abopt.abopt2 import LBFGS, TrustRegionCG

from MADLens.lightcone import *
from nbodykit.cosmology import Planck15, LinearPower
import scipy
from mpi4py import MPI
from absl import app
from absl import flags
import subprocess

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

from abopt.abopt2 import LBFGS, TrustRegionCG
import numpy as np

In [3]:
randseed = 123
flags.DEFINE_float('noise_variance',0.1,'noise std')
flags.DEFINE_float('boxsize',32.,'size of the simulation box in Mpc/h')
flags.DEFINE_integer('Nmesh',32,'resolution of fastPM mesh')
flags.DEFINE_integer('Nmesh2D',256, 'resolution of lensing map')
flags.DEFINE_float('boxsize2D',6.37616,'field of view in degrees (default is optimal for default settings, use FindConfigs.ipynb notebook to find optimal fov for your setting.')
flags.DEFINE_integer('N_steps',11,'number of fastPM steps')
#bounds from KIDS contours, default values from Planck2015
flags.DEFINE_float('Omega_m',0.3089,'total matter density', lower_bound=0.1, upper_bound=0.5)
flags.DEFINE_float('sigma_8',0.8158,'amplitude of matter fluctuations', lower_bound=0.4, upper_bound=1.3)
flags.DEFINE_boolean('PGD',False,'whether to use PGD sharpening')
flags.DEFINE_integer('B',2,'force resolution factor')
flags.DEFINE_spaceseplist('zs_source',['0.1','0.2'],'source redshifts')
flags.DEFINE_boolean('interpolate',False,'whether to interpolate between snapshots')
flags.DEFINE_boolean('debug',True,'debug mode allows to run repeatedly with the same settings')
flags.DEFINE_boolean('save3D',False,'whether to dump the snapshots, requires interp to be set to False')
flags.DEFINE_boolean('save3Dpower', False, 'whether to measure and save the power spectra of the snapshots')
flags.DEFINE_enum('mode', 'backprop', ['forward','backprop'],'whether to run the forward model only or include backpropagation')
flags.DEFINE_boolean('analyze', False, 'whether to print out resource usage')
flags.DEFINE_string('label', 'test_run', 'label of this run')

In [4]:
FLAGS = flags.FLAGS
params              = FLAGS.flag_values_dict()
params['Nmesh']     = [params['Nmesh']]*3
params['BoxSize']   = [params['boxsize']]*3
params['Nmesh2D']   = [params['Nmesh2D']]*2
params['BoxSize2D'] = [params['boxsize2D']]*2
params['zs_source'] = [float(zs) for zs in params['zs_source']]
cosmo = Planck15.match(Omega0_m=params['Omega_m'])
cosmo = cosmo.match(sigma8=params['sigma_8'])

In [5]:
def convolve(power_spectrum, field):
    """ Convolve a field with a power spectrum.
        Preserve the type of the output field.
    """
    c = field.cast(type='complex')
    c = c.apply(lambda k, v: (power_spectrum(k.normp() ** 0.5) / c.pm.BoxSize.prod()) ** 0.5 * v)
    return c.cast(type=type(field))

In [6]:
def generate_ics(pm, randseed, cosmo):
    
    rho       = pm.generate_whitenoise(seed=randseed, unitary=False, type='complex')
    rho       = rho.apply(lambda k, v:(cosmo.get_pklin(k.normp(2) ** 0.5, 0) / pm.BoxSize.prod()) ** 0.5 * v)
    #set zero mode to zero
    rho.csetitem([0, 0, 0], 0)
    
    return rho

In [17]:
pm        = fastpm.ParticleMesh(Nmesh=params['Nmesh'], BoxSize=params['BoxSize'], comm=MPI.COMM_WORLD, resampler='cic')

    # 2D FOV in radians
BoxSize2D = [deg/180.*np.pi for deg in params['BoxSize2D']]

np.random.seed(randseed)
randseeds = np.random.randint(0,1e6,100)

    # generate initial conditions
cosmo     = cosmo.clone(P_k_max=30)


    # weak lensing simulation object
wlsim     = WLSimulation(stages = numpy.linspace(0.1, 1.0, params['N_steps'], endpoint=True), cosmology=cosmo, pm=pm, boxsize2D=BoxSize2D, params=params)

model     = wlsim.run

number of box replications required to fill field of view: 2.0
insufficient number of rotations to fill lightcone


In [8]:
@autooperator('nl, s, fs->y')
def PriorOperator( nl, s, fs, invS):
    # when |s|^2 and invS are the same, this is supposed
    # to return 1.0
    fac = s.eval("x.pm.Nmesh.prod() ** -0.5")
    fac = fac * (invS ** 0.5)
    s_over_Shalf = s * fac
    r = fastpm.c2r(s_over_Shalf)
    return dict(y = r)


@autooperator('nl, s, fs -> y')
def NLResidualOperator(nl, s, fs, d, invvar):
    r = linalg.add(fs, d * -1)
    r = linalg.mul(r, invvar ** 0.5)
    return dict(y = r)

In [9]:
class FastPMVectorSpace(MPIVectorSpace):
    def dot(self, a, b):
        """ einsum('i,i->', a, b) """
        if isinstance(a, Field):
            return a.cdot(b).real
        else:
            return self.comm.allreduce(numpy.sum(a * b))

class ChiSquareProblem(MPIChiSquareProblem):
    ComplexOptimizer = Preconditioner(Pvp=lambda x, direction: x, vPp=lambda x, direction: x)

    RealOptimizer = Preconditioner(
        Pvp=lambda x, direction:
            x.c2r() / x.Nmesh.prod() if direction > 0 else x.c2r(),
        vPp=lambda x, direction:
            x.r2c() if direction > 0 else x.r2c() * x.Nmesh.prod()
    )

    def __init__(self, comm, forward_operator, residuals):
        vs = FastPMVectorSpace(comm)
        MPIChiSquareProblem.__init__(self, comm, forward_operator, residuals, vs)


In [10]:
def minimize(problem, s_ini_, data_, mini):
    """
    s_ini_: initial guess
    data_ : input data
    mini  : minimizer
    """
    
    def monitor(state):
        log.print('time',state['wallclock'],'nit:',state['nit'],'gnorm:',state['gnorm'],'y:',state['y'],'dy:',state['dy'],'fev:',state['fev'],'converged:',state['converged'],'message:',state['message'])
        if state['dxnorm'] is not None:
            log.fileprint('%d %f %f %f %f %f %d %d %d'%(state['nit'],state['gnorm'],state['y'],state['dy'],state['xnorm'],state['dxnorm'],state['converged'],state['fev'],state['gev']))
        if state['nit'] in settings['dump_iter'] or state['converged']==True:
            print(state['nit'],state['nit'] in settings['dump_iter'],state['converged']==True)

    r = mini.minimize(problem, s_ini_, monitor=monitor)

In [11]:
def generate_noise(noise_variance):
    """
    noise_variance
    """
    pass

In [ ]:
s_true  = generate_ics(pm, randseeds[0], cosmo)

fs      = model.build().compute(vout='kmaps', init=dict(rhok=s_true),return_tape=False)

S_fid   = convolve(LinearPower(cosmo,0), pm.create(type='complex', value=1)) ** 2

#noise  = generate_noise
data    = fs #+noise

In [14]:
print(model)

problem = MPIChiSquareProblem(pm.comm,model,
            [
                PriorOperator.bind(invS=S_fid ** -1),
                NLResidualOperator.bind(d=data, invvar=params['noise_variance'] ** -1)
            ],
)


s_true  = generate_ics(pm, randseeds[0], cosmo)

s_ini  = generate_ics(pm, randseeds[1], cosmo)*1e-4

minimizer = LBFGS(maxiter=50,m=10, conviter=3)

minimize(problem,s_ini,data,minimizer)




TypeError: AutoOperator object argument after * must be an iterable, not Symbol